In [1]:

%%writefile omp.cpp
#include<iostream>
#include<omp.h>
#include<vector>
#include<queue>
#include<stack>
#include <cstdlib>
#include <ctime>
#define max_nodes 100
using namespace std;

class Graph{
    int numnodes;
    vector<vector<int>> adj;

public:
    Graph(int x) : numnodes(x), adj(x) {}

    void addedge(int u,int v){
        adj[u].push_back(v);
        adj[v].push_back(u);
        return;
    }

    void BFS(int start){
        cout<<"BFS ";
        vector<bool> visited(numnodes,false);
        queue<int> q;

        visited[start] = true;
        q.push(start);

        #pragma omp parallel
        {
            while(true) {
                int u = -1; // Initialize u to an invalid value
                #pragma omp critical (QueuePop)
                {
                    if (!q.empty()) {
                        u = q.front();
                        q.pop();
                    }
                }

                if (u == -1) break; // Exit if the queue is empty

                for (int v: adj[u]){
                    if (!visited[v]){
                        #pragma omp parallel
                        visited[v] = true;
                        #pragma omp critical (QueuePush)
                        {
                            q.push(v);
                        }
                    }
                }
            }
        }

        cout << "BFS from vertex " << start << ":\n";
        for (int i = 0; i < numnodes; i++) {
            if (visited[i]) cout << i << " ";
        }
        cout << endl;
    }

    void DFS(int start){
        cout<<"DFS ";
        vector<bool> visited(numnodes,false);
        stack<int> s;

        visited[start] = true;
        s.push(start);

        #pragma omp parallel
        {
            while(true){
                int u = -1; // Initialize u to an invalid value
                #pragma omp critical (StackPop)
                {
                    if (!s.empty()){
                        u = s.top();
                        s.pop();
                    }
                }

                if (u == -1) break; // Exit if the stack is empty

                for(int v: adj[u]){
                    if(!visited[v]){
                        #pragma omp parallel
                        visited[v] = true;
                        #pragma omp critical (StackPush)
                        {
                            s.push(v);
                        }
                    }
                }
            }
        }

        cout << "DFS from vertex " << start << ":\n";
        for (int i = 0; i < numnodes; i++) {
            if (visited[i]) cout << i << " ";
        }
        cout << endl;
    }
};

int main() {
    srand(time(NULL));
    Graph g(50);

    for (int i = 0; i < 50; ++i) {
        int n = rand() % 50 + 1;
        int m = rand() % 50 + 1;
        g.addedge(n,m);
    }

    int start = rand() % 50 + 1;

    cout << "Parallel BFS:\n";
    g.BFS(start);

    cout << "\nParallel DFS:\n";
    g.DFS(start);
    return 0;
}


Writing omp.cpp


In [2]:
%%script bash
g++ -fopenmp omp.cpp -o omp

In [3]:
!./omp

In [7]:
%%writefile opm2.cpp
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <omp.h>

// Sequential Bubble Sort
void bubbleSort(int arr[], int n) {
    for (int i = 0; i < n-1; i++) {
        for (int j = 0; j < n-i-1; j++) {
            if (arr[j] > arr[j+1]) {
                std::swap(arr[j], arr[j+1]);
            }
        }
    }
}

// Parallel Bubble Sort
void parallelBubbleSort(int arr[], int n) {
    #pragma omp parallel
    {
        for (int i = 0; i < n-1; i++) {
            #pragma omp for
            for (int j = 0; j < n-i-1; j++) {
                if (arr[j] > arr[j+1]) {
                    std::swap(arr[j], arr[j+1]);
                }
            }
        }
    }
}

// Merge function for merge sort
void merge(int arr[], int l, int m, int r) {
    int n1 = m - l + 1;
    int n2 = r - m;

    int L[n1], R[n2];
    for (int i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (int j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    int i = 0;
    int j = 0;
    int k = l;

    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }
}

// Sequential Merge Sort
void mergeSort(int arr[], int l, int r) {
    if (l >= r) return;

    int m = l + (r - l) / 2;
    mergeSort(arr, l, m);
    mergeSort(arr, m + 1, r);
    merge(arr, l, m, r);
}

// Parallel Merge Sort
void parallelMergeSort(int arr[], int l, int r) {
    if (l >= r) return;

    int m = l + (r - l) / 2;
    #pragma omp parallel sections
    {
        #pragma omp section
        parallelMergeSort(arr, l, m);
        #pragma omp section
        parallelMergeSort(arr, m + 1, r);
    }
    merge(arr, l, m, r);
}

// Print array
void printArray(int arr[], int size) {
    for (int i = 0; i < size; i++) {
        std::cout << arr[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    const int SIZE = 10;
    int arr1[SIZE], arr2[SIZE];

    // Initialize arrays with random values
    srand(time(NULL));
    for (int i = 0; i < SIZE; i++) {
        arr1[i] = rand() % 100;
        arr2[i] = arr1[i];
    }

    std::cout << "Original Array: ";
    printArray(arr1, SIZE);

    // Sequential Bubble Sort
    bubbleSort(arr1, SIZE);
    std::cout << "Sequential Bubble Sort: ";
    printArray(arr1, SIZE);

    // Parallel Bubble Sort
    parallelBubbleSort(arr2, SIZE);
    std::cout << "Parallel Bubble Sort: ";
    printArray(arr2, SIZE);

    // Reset arrays
    for (int i = 0; i < SIZE; i++) {
        arr1[i] = arr2[i] = rand() % 100;
    }

    std::cout << "Original Array: ";
    printArray(arr1, SIZE);

    // Sequential Merge Sort
    mergeSort(arr1, 0, SIZE - 1);
    std::cout << "Sequential Merge Sort: ";
    printArray(arr1, SIZE);

    // Parallel Merge Sort
    parallelMergeSort(arr2, 0, SIZE - 1);
    std::cout << "Parallel Merge Sort: ";
    printArray(arr2, SIZE);

    return 0;
}


Writing opm2.cpp


In [8]:
%%script bash
g++ -fopenmp opm2.cpp -o omp2

In [9]:
!./omp2

Original Array: 61 97 17 81 3 37 89 8 48 31 
Sequential Bubble Sort: 3 8 17 31 37 48 61 81 89 97 
Parallel Bubble Sort: 3 8 17 31 37 48 61 81 89 97 
Original Array: 37 36 52 59 81 18 95 70 7 97 
Sequential Merge Sort: 7 18 36 37 52 59 70 81 95 97 
Parallel Merge Sort: 7 18 36 37 52 59 70 81 95 97 
